# Making FC Barcelona Face Recognition Model with VGG16

#### Loading the VGG16 Model

In [1]:
from keras.applications import VGG16

# Setting the input size now to 64 x 64 pixel 
img_rows = 64
img_cols = 64 

# loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers, i.e. include_top = False

Using TensorFlow backend.


#### Freezing all the layers except the Top 4

In [2]:
# Layers are set to trainable as True by default
for layer in vgg16.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


#### Let's make a function that returns our FC Head

In [3]:
def barcaface(bottom_model, num_classes):
  """creates the top or head of the model that will be 
    placed on top of the bottom layers"""
  top_model = bottom_model.output
  top_model = Flatten(name = "flatten")(top_model)
  top_model = Dense(512, activation='relu')(top_model)
  top_model = Dense(256, activation='relu')(top_model)
  top_model = Dense(128, activation='relu')(top_model)
  top_model = Dense(num_classes, activation='softmax')(top_model)
  return top_model

#### Let's add our FC head back onto VGG16

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
    
# Number of classes in the Flowers-17 dataset
num_classes = 3

FC_Head = barcaface(vgg16, num_classes)

model = Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

#### Loading our Barca Dataset

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Train/'
validation_data_dir = 'Test/'

# Augmenting the data!
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 324 images belonging to 3 classes.
Found 100 images belonging to 3 classes.


#### Training out our Model

* We are using the concept of checkpoint and early stopping here!

In [6]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
                   
checkpoint = ModelCheckpoint("FCBarcelona_vgg16_64.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.00001)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint, reduce_lr]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.0001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 324
nb_validation_samples = 100

# We are only training 6 EPOCHS
epochs = 10
batch_size = 5

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

# Saving our model
model.save("FCBarcelona_vgg16_64.h5")

Epoch 1/10
64/64 [==============================] - 123s 2s/step - loss: 1.0486 - accuracy: 0.4453 - val_loss: 1.0313 - val_accuracy: 0.4000

Epoch 00001: val_loss improved from inf to 1.03129, saving model to FCBarcelona_vgg16_64.h5
Epoch 2/10
64/64 [==============================] - 115s 2s/step - loss: 0.9435 - accuracy: 0.5668 - val_loss: 1.1568 - val_accuracy: 0.4700

Epoch 00002: val_loss did not improve from 1.03129
Epoch 3/10
64/64 [==============================] - 113s 2s/step - loss: 0.8707 - accuracy: 0.6053 - val_loss: 0.6617 - val_accuracy: 0.4400

Epoch 00003: val_loss improved from 1.03129 to 0.66174, saving model to FCBarcelona_vgg16_64.h5
Epoch 4/10
64/64 [==============================] - 112s 2s/step - loss: 0.8073 - accuracy: 0.6373 - val_loss: 0.7634 - val_accuracy: 0.4800

Epoch 00004: val_loss did not improve from 0.66174
Epoch 5/10
64/64 [==============================] - 102s 2s/step - loss: 0.7836 - accuracy: 0.6490 - val_loss: 0.5603 - val_accuracy: 0.4000



#### Loading our classifier

In [7]:
from keras.models import load_model

classifier = load_model('FCBarcelona_vgg16.h5')

#### Testing our classifier on some test images

In [11]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

fc_barcelona_dict = {"[0]": "De Jong",
                    "[1]": "Messi",
                    "[2]": "Gerard Pique",}

fc_barcelona_dict_n = {"De_Jong": "De Jong",
                    "Messi": "Messi",
                    "Pique": "Gerard Pique",}

def draw_test(name, pred, im):
    player = fc_barcelona_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, player, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + fc_barcelona_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("Test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - Gerard Pique
Class - De Jong
Class - Messi
Class - Gerard Pique
Class - Gerard Pique
Class - De Jong
Class - Messi
Class - Messi
Class - Messi
Class - Gerard Pique
